<a href="https://colab.research.google.com/github/MSFTserver/AI-Colab-Notebooks/blob/main/NNST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Neural Neighbor Style Transfer 
https://github.com/nkolkin13/NeuralNeighborStyleTransfer
<br><br>

Colab ~~Repo~~ for the algorithm NNST-Opt, described in the preprint "Neural Neighbor Style Transfer", please feel free to email any questions to kolkin@adobe.com Paper Link: https://ttic.uchicago.edu/~nickkolkin/Paper/NNST_Preprint.pdf

----
<br><br>
made to run in colab by [MSFTserver](https://gist.github.com/MSFTserver) aka [HostsServer](https://twitter.com/hostsserver)

In [ ]:
#@markdown # 1) Download & Import Dependecies

# Core Imports
import time
import argparse
import random
import sys

# External Dependency Imports
from imageio import imwrite
import torch
import numpy as np

# Internal Project Imports
!git clone https://github.com/nkolkin13/NeuralNeighborStyleTransfer
sys.path.append('/content/NeuralNeighborStyleTransfer')
from pretrained.vgg import Vgg16Pretrained
from utils import misc as misc
from utils.misc import load_path_for_pytorch
from utils.stylize import produce_stylization

In [ ]:
#@markdown # 2) Prepare Folders
import os
from google.colab import drive

def createPath(filepath):
    if os.path.exists(filepath) == False:
      os.makedirs(filepath)
      print(f'Made {filepath}')
    else:
      print(f'filepath {filepath} exists.')
#@markdown optionally link google drive and upload images to `AI/NNST/in` folder
google_drive = False #@param {type:"boolean"}

if google_drive:
  print("Using Google Drive.")
  drive.mount('/content/drive')
  root_path = '/content/drive/MyDrive/AI/NNST'
else:
  google_drive = False
  print("Google Drive not connected.")
  root_path = '/content'

input_path = f'{root_path}/in'
output_path = f'{root_path}/out'

createPath(input_path)
createPath(output_path)

In [ ]:
#@markdown # 3) Upload Content Image
#@markdown ### <font color="red"> READ BEFORE RUNNING </font>
#@markdown enter the image name including extension before running!
#@markdown #### &nbsp;&nbsp;&nbsp;- <font color="green">Google Drive Mounted: </font>
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
#@markdown => if you linked Google Drive please upload images to the `AI/NNST/in` folder that was created.

#@markdown #### &nbsp;&nbsp;&nbsp;- <font color="blue">No Google Drive Mounted: </font>
#@markdown &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
#@markdown => if you didnt upload to google drive proceed to running this cell and uploading your image 
#@markdown <br><br>
content_image_file_name = '!!!DONT USE IF NOT MOUNTING DRIVE!!!' #@param {type:"string"}

import os
from google.colab import files
import shutil

if google_drive:
  content_path = f'{input_path}/{content_image_file_name}'
else:
  uploaded_content = files.upload()
  content_dst_path = os.path.join(input_path, list(uploaded_content.keys())[0])
  shutil.move(list(uploaded_content.keys())[0], content_dst_path)
  content_path = f'{input_path}/{list(uploaded_content.keys())[0]}'

from IPython.display import Image
Image(content_path, height=500)

In [ ]:
#@markdown # 4) Upload Style Image
#@markdown ## <font color="red"> READ BEFORE RUNNING </font>
#@markdown ### <font color="green">Google Drive Mounted: </font>

#@markdown => enter the image name including extension before running!<br>
#@markdown => if you linked Google Drive please upload images to the `AI/NNST/in` folder that was created.
#@markdown #### <font color="blue">No Google Drive Mounted: </font>

#@markdown => if you didnt upload to google drive proceed to running this cell and uploading your image 
#@markdown <br><br>
style_image_file_name = '!!!DONT USE IF NOT MOUNTING DRIVE!!!' #@param {type:"string"}

import os
from google.colab import files
import shutil

if google_drive:
  style_path = f'{input_path}/{style_image_file_name}'
else:
  uploaded_style = files.upload()
  style_dst_path = os.path.join(input_path, list(uploaded_style.keys())[0])
  shutil.move(list(uploaded_style.keys())[0], style_dst_path)
  style_path = f'{input_path}/{list(uploaded_style.keys())[0]}'

from IPython.display import Image
Image(style_path, height=500)

In [ ]:
#@markdown # 5) Config
# Fix Random Seed
random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

# Define Configuration
high_res = False #@param {type:"boolean"}
cpu = False #@param {type:"boolean"}
no_flip = False #@param {type:"boolean"}
content_loss = False #@param {type:"boolean"}
dont_colorize = False #@param {type:"boolean"}
alpha=0.75 #@param {type:"number"}

# Interpret config options arguments
max_scls = 4
sz = 512
if high_res:
    max_scls = 5
    sz = 1024
flip_aug = (not no_flip)
misc.USE_GPU = (not cpu)
content_weight = 1. - alpha

# Error checking for arguments
# error checking for paths deferred to imageio
assert (0.0 <= content_weight) and (content_weight <= 1.0), "alpha must be between 0 and 1"
assert torch.cuda.is_available() or (not misc.USE_GPU), "attempted to use gpu when unavailable"

In [ ]:
#@markdown # 6) Run Style Transfer
# Define feature extractor
cnn = misc.to_device(Vgg16Pretrained())
phi = lambda x, y, z: cnn.forward(x, inds=y, concat=z)

# Load images
content_im_orig = misc.to_device(load_path_for_pytorch(content_path, target_size=sz)).unsqueeze(0)
style_im_orig = misc.to_device(load_path_for_pytorch(style_path, target_size=sz)).unsqueeze(0)

# Run Style Transfer
torch.cuda.synchronize()
start_time = time.time()
output = produce_stylization(content_im_orig, style_im_orig, phi,
                            max_iter=200,
                            lr=2e-3,
                            content_weight=content_weight,
                            max_scls=max_scls,
                            flip_aug=flip_aug,
                            content_loss=content_loss,
                            dont_colorize=dont_colorize)
torch.cuda.synchronize()
print('Done! total time: {}'.format(time.time() - start_time))

# Convert from pyTorch to numpy, clip to valid range
new_im_out = np.clip(output[0].permute(1, 2, 0).detach().cpu().numpy(), 0., 1.)

# Save stylized output
save_im = (new_im_out * 255).astype(np.uint8)
imwrite(f'{output_path}/output.jpg', save_im)

# Free gpu memory in case something else needs it later
if misc.USE_GPU:
    torch.cuda.empty_cache()

from IPython.display import Image
Image(f'{output_path}/output.jpg')